In [72]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL",
          "IN", "IA", "KS", "KY", "LA", "ME",
         "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV",
          "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA",
         "RI", "SC", "SD", "TN", "TX", "UT", "VT", 
          "VA", "WA", "WV", "WI", "WY"]

In [77]:
# Get the State Data
sites = []
for state in states:
    url = "https://www.naccho.org/membership/lhd-directory?searchType=standard&lhd-search=&lhd-state=%s" % state

    res = requests.get(url)
    assert res.status_code == 200
    
    sites.append(res.text)

In [46]:
"southwest" in soup.text.lower()

True

In [80]:
len(sites)

50

In [83]:
data = {
    "state": [],
    "public_health": [],
    "phone_number": []
    
}
cnt = 0
for s in range(0, len(sites)):
    soup = BeautifulSoup(sites[s], "html.parser")
    if soup == None:
        continue
    state = states[s]
    # Find all the data
    for td in soup.table.find_all("td"):
        if td == None:
            continue
        if "health" in td.text.lower():
            # if cnt == 1, means last one didn't have a phone number, so pop it
            if cnt == 1:
                data["state"].pop()
                data["public_health"].pop()
                cnt = 0
                
            data["state"].append(state)
            data["public_health"].append(td.text)
            cnt=1
        
        if "phone" in td.text.lower():
            # Update the data with corresponding phone number
            # Updates only if previous update was 
            if cnt == 1:
                if td.a == None:
                    continue
                num = td.a.get("href")
                data["phone_number"].append(num[5:])
                cnt = 0
                
        
            
assert len(data["public_health"]) == len(data["phone_number"])

In [85]:
pd.DataFrame(data).to_csv("../data/all_state_numbers.csv")